# Build a RAG pipeline

Create a retrieval-augmented generation system that answers questions using your documents as context.


## Problem

You want an LLM to answer questions using your specific documents—not just its training data. You need to retrieve relevant context and include it in the prompt.

| Use case | Documents | Questions |
|----------|-----------|-----------|
| Customer support | Help articles | "How do I reset my password?" |
| Internal wiki | Company docs | "What's our vacation policy?" |
| Research | Papers | "What did the study find about X?" |


## Solution

**What's in this recipe:**
- Embed and index documents for retrieval
- Create a query function that retrieves context
- Generate answers grounded in your documents

You build a pipeline that: (1) embeds documents, (2) finds relevant chunks for a query, and (3) generates an answer using those chunks as context.


### Setup


In [ ]:
%pip install -qU pixeltable openai


In [ ]:
import os
import getpass

if 'OPENAI_API_KEY' not in os.environ:
    os.environ['OPENAI_API_KEY'] = getpass.getpass('OpenAI API Key: ')


In [ ]:
import pixeltable as pxt
from pixeltable.functions.openai import embeddings, chat_completions


In [ ]:
# Create a fresh directory
pxt.drop_dir('rag_demo', force=True)
pxt.create_dir('rag_demo')


### Step 1: Create document store with embeddings


In [ ]:
# Create table for document chunks
chunks = pxt.create_table(
    'rag_demo.chunks',
    {'doc_id': pxt.String, 'chunk_text': pxt.String}
)


In [ ]:
# Add embedding column
chunks.add_computed_column(
    embedding=embeddings(chunks.chunk_text, model='text-embedding-3-small')
)


In [ ]:
# Create embedding index for fast retrieval
chunks.add_embedding_index('embedding', metric='cosine')


### Step 2: Load documents


In [ ]:
# Sample knowledge base (in production, load from files/database)
documents = [
    {
        'doc_id': 'password-reset',
        'chunk_text': 'To reset your password, go to the login page and click "Forgot Password". Enter your email address and you will receive a reset link within 5 minutes. The link expires after 24 hours.'
    },
    {
        'doc_id': 'password-reset',
        'chunk_text': 'Password requirements: minimum 8 characters, at least one uppercase letter, one number, and one special character. Passwords expire every 90 days for security.'
    },
    {
        'doc_id': 'account-settings',
        'chunk_text': 'To update your profile, navigate to Settings > Account. You can change your display name, email address, and notification preferences. Changes take effect immediately.'
    },
    {
        'doc_id': 'billing',
        'chunk_text': 'Billing occurs on the first of each month. You can view invoices under Settings > Billing. To change your payment method, click "Update Payment" and enter your new card details.'
    },
    {
        'doc_id': 'api-access',
        'chunk_text': 'API keys can be generated in Settings > Developer. Each key has configurable permissions. Rate limits are 1000 requests per minute for standard plans, 10000 for enterprise.'
    },
]

chunks.insert(documents)


### Step 3: Create the RAG query function


In [ ]:
# Define a query function that retrieves context and generates answer
@pxt.query
def retrieve_context(query: str, top_k: int = 3) -> list[dict]:
    """Retrieve the most relevant chunks for a query."""
    results = chunks.order_by(
        chunks.embedding.similarity(query),
        asc=False
    ).limit(top_k).select(chunks.doc_id, chunks.chunk_text).collect()
    return [{'doc_id': r['doc_id'], 'text': r['chunk_text']} for r in results]


In [ ]:
# Test retrieval
query = "How do I reset my password?"
context_chunks = retrieve_context(query)

print(f"Query: {query}\n")
print("Retrieved context:")
for i, chunk in enumerate(context_chunks, 1):
    print(f"  {i}. [{chunk['doc_id']}] {chunk['text'][:80]}...")


### Step 4: Generate answers with context


In [ ]:
# Create a table for questions/answers
qa = pxt.create_table(
    'rag_demo.qa',
    {'question': pxt.String}
)


In [ ]:
# Add retrieval step
qa.add_computed_column(context=retrieve_context(qa.question, top_k=3))


In [ ]:
# Build the RAG prompt
@pxt.udf
def build_rag_prompt(question: str, context: list[dict]) -> str:
    context_text = '\n\n'.join([f"[{c['doc_id']}]: {c['text']}" for c in context])
    return f"""Answer the question based only on the provided context. If the context doesn't contain the answer, say "I don't have information about that."

Context:
{context_text}

Question: {question}

Answer:"""

qa.add_computed_column(prompt=build_rag_prompt(qa.question, qa.context))


In [ ]:
# Generate answer
qa.add_computed_column(
    response=chat_completions(
        messages=[{'role': 'user', 'content': qa.prompt}],
        model='gpt-4o-mini'
    )
)
qa.add_computed_column(answer=qa.response.choices[0].message.content)


### Ask questions


In [ ]:
# Insert questions
questions = [
    {'question': 'How do I reset my password?'},
    {'question': 'What are the API rate limits?'},
    {'question': 'When am I billed?'},
]

qa.insert(questions)


In [ ]:
# View answers
for row in qa.select(qa.question, qa.answer).collect():
    print(f"Q: {row['question']}")
    print(f"A: {row['answer']}\n")


## Explanation

**RAG pipeline flow:**

```
Question → Embed → Retrieve similar chunks → Build prompt with context → Generate answer
```

**Key components:**

| Component | Purpose |
|-----------|---------|
| Embedding index | Fast similarity search |
| `@pxt.query` | Retrieve context from the database |
| `@pxt.udf` | Build the augmented prompt |
| Computed columns | Chain the pipeline together |

**Scaling tips:**

- Use `doc-chunk-for-rag` recipe to split long documents
- Adjust `top_k` to balance context size vs. relevance
- Consider metadata filtering for large knowledge bases


## See also

- [Chunk documents for RAG](https://docs.pixeltable.com/howto/cookbooks/text/doc-chunk-for-rag) - Split documents into chunks
- [Create text embeddings](https://docs.pixeltable.com/howto/cookbooks/search/embed-text-openai) - Embedding fundamentals
- [Semantic text search](https://docs.pixeltable.com/howto/cookbooks/search/search-semantic-text) - Search patterns
